Set your API keys as env variables:

```bash
echo "export OPENAI_API_KEY='yourkey'" >> ~/.bashrc
echo "export CLASIFAI_OPENAI_ORG_ID='yourid'" >> ~/.bashrc
source ~/.bashrc
# To use in remote jupyter, need to kill and restart with "Kill VS Code Server On Host"
#  https://stackoverflow.com/a/72983479
```

In [1]:
import os
import json

import pandas as pd
import tiktoken
from openai import OpenAI

In [2]:
api_key = os.environ["OPENAI_API_KEY"]
org_id = os.environ["CLASIFAI_OPENAI_ORG_ID"]

In [3]:
client = OpenAI(api_key=api_key, organization=org_id)

In [4]:
# all cols as str:
df_items = pd.read_csv("~/Downloads/items.csv", dtype=str)

In [5]:
# TODO no se guardaron los 0s a la izquierda?
# df_items.query("nomenclatura.astype('str').str.startswith('01')")

In [6]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0125"):
    """Return the number of tokens used by a list of messages.
    Source: https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken#6-counting-tokens-for-chat-completions-api-calls
    """
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0125",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens


def get_completion(
    client: OpenAI,
    messages: list[dict[str, str]],
    model: str = "gpt-3.5-turbo-0125",
    max_tokens=500,
    temperature=1,  # defaults to 1 (https://platform.openai.com/docs/api-reference/chat/create#chat-create-temperature)
    stop=None,
    seed=33,
    response_format: str = "text", # Setting to "json_object" enables JSON mode
    logprobs=None,  # whether to return log probabilities of the output tokens or not. If true, returns the log probabilities of each output token returned in the content of message..
    top_logprobs=None,
    tools=None,
) -> str:
    """
    messages: list like
    [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
    NOTE Including conversation history is important when user instructions 
    refer to prior messages ... Because the models have no memory of past
    requests, all relevant information must be supplied as part of the
    conversation history in each request. If a conversation cannot fit
    within the model’s token limit, it will need to be shortened in some way.
    Source: https://platform.openai.com/docs/guides/text-generation/chat-completions-api
    -------
    Source: https://cookbook.openai.com/examples/using_logprobs#0-imports-and-utils
    For other params see: https://platform.openai.com/docs/api-reference/chat/create 
    """
    n_tokens = num_tokens_from_messages(
        messages,
        # [{"role": "user", "content": prompt}]
    )
    if n_tokens > 1200:
        print(f"Skipping chunk with {n_tokens} tokens")
        return
    params = {
        "model": model,
        "messages": messages,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "stop": stop,
        "seed": seed,
        "logprobs": logprobs,
        "top_logprobs": top_logprobs,
        "response_format": {"type": response_format},
    }
    if tools:
        params["tools"] = tools
    completion = client.chat.completions.create(**params)
    return completion


def get_code_notes(df: pd.DataFrame, code: str) -> str:
    df_code = df.query("nomenclatura.astype('str') == @code")
    if df_code.empty:
        print("No se encontró el código")
        return
    elif df_code.shape[0] > 1:
        print("Se encontraron múltiples códigos")
        return
    cols = [c for c in df_items.columns if c.startswith("notas")]
    res = (
        df_code[cols]
            .rename(columns=lambda x: x.replace("notas_", ""))
            .fillna("")
            .map(lambda x: x.strip())
            .map(lambda x: x.capitalize())
            .to_dict(orient="records")
    )[0]
    return res


def get_code_descriptions(df: pd.DataFrame, code: str) -> dict:
    df_code = df.query("nomenclatura.astype('str') == @code")
    if df_code.empty:
        print("No se encontró el código")
        return
    elif df_code.shape[0] > 1:
        print("Se encontraron múltiples códigos")
        return
    cols = [c for c in df_items.columns if c.startswith("descripcion")]
    res = (
        df_code[cols]
            .rename(columns=lambda x: x.replace("descripcion_", ""))
            .fillna("")
            .apply(lambda x: x.str.replace(r"\.$", "", regex=True))
            .map(lambda x: x.strip())
            .map(lambda x: x.capitalize())
            .to_dict(orient="records")
    )[0]
    return res
    

def get_first_last_capitulos(df: pd.DataFrame, seccion_desc: str) -> tuple[str, str]:
    """Return the first and last chapter of a section."""
    mask = df["descripcion_seccion"].str.contains(seccion_desc, case=False)
    df_seccion = df[mask]
    capitulos = df_seccion["nomenclatura"].astype(str).str[:2].sort_values().unique()
    capitulo_first, capitulo_last = capitulos[0], capitulos[-1]
    return capitulo_first, capitulo_last


In [7]:
# Input:
item_description = (
    # "CALZADO CON RUEDAS PARA JOVENES, CONSTITUIDO POR UN CALZADO TIPO BOTA CON"
    # " REFUERZO PLASTICO EN LOS TOBILLOS, PROVISTO CADA UNO DE CUATRO RUEDAS FIJAS"
    # " EN LINEA. 80% PLASTICO, 20% CUERO. CALZADO PARA LA PRACTICA DE PATINAJE SOBRE RUEDAS."
    
    "LIMONES FRESCOS A GRANEL. FRESCOS." # NOTE en este ejemplo pasan "limones" como "funcionalidad"
    # NOTE aca chatgpt responde correctamente en base a conocimiento previo.
    # La respuesta es buena pero la justificacion es mala.

    # "GRABADOR Y SERVIDOR DE VIDEO PARA TRASMISION DE TELEVISION Y REPRODUCCION."
    # " APARATO DE GRABACION, CAPAZ DE REPRODUCIR IMAGEN Y SONIDO EN FORMA SIMULTANEA"
    # " O PROGRAMADA, CON CAPACIDAD DE GRABACION DE HASTA 100 HS EN 4 DISCOS RIGIDOS"
    # " DE 250GB CADA UNO, CON 1 ENTRADA Y 3 SALIDAS DE VIDEO COMPUESTO, 2 CANALES"
    # " DE AUDIO MONO PARA CADA UNA, 2 ENTRADAS/SALIDAS DE AUDIO ESTEREO DIGITAL Y 5 PUERTOS DE CONTROL."
    
    # "CABALLO DE CIRCO."

)

# Candidate data:
# candidate_code = "63079090"
candidate_code = "33011300"
# candidate_code = "84717020"
# candidate_code = "01012100"

# TODO automatizar, o tal vez no es necesario usar este nro.
# seccion = "XI"
seccion = "VI"
# seccion = "XVI"
# seccion = "I" 

# capitulo, partida, subpartida, posicion = [candidate_code[i:i+2] for i in range(0, len(candidate_code), 2)]
# candidate_description = get_code_description(seccion)
# candidate_notes = get_code_notes(seccion)

candidate_descriptions = get_code_descriptions(df_items, candidate_code)
candidate_notes = get_code_notes(df_items, candidate_code)

seccion_description = candidate_descriptions.get("seccion", "")
seccion_notes = candidate_notes.get("seccion", "")
first_capitulo, last_capitulo = get_first_last_capitulos(df_items, seccion_description)

In [8]:
print(candidate_descriptions)
print(candidate_notes)

# TODO corregir algunas notas e.g. "no comprende:los artículos " -> "No comprende: los artículos"
# TODO corregir notas e.g. notas de seccion XI (Materias textiles y sus manufacturas)
# TODO agregar automaticamente descripciones entre parentesis en las notas

{'seccion': 'Productos de las industrias químicas o de las industrias conexas', 'capitulo': 'Aceites esenciales y resinoides; preparaciones de perfumería, de tocador o de cosmética', 'partida': 'Aceites esenciales (desterpenados o no), incluidos los «concretos» o «absolutos»; resinoides; oleorresinas de extracción; disoluciones concentradas de aceites esenciales en grasas, aceites fijos, ceras o materias análogas, obtenidas por enflorado o maceración; subproductos terpénicos residuales de la desterpenación de los aceites esenciales; destilados acuosos aromáticos y disoluciones acuosas de aceites esenciales', 'subpartida_1er': 'Aceites esenciales de agrios (cítricos):', 'subpartida_2do': '', 'item_1er': '', 'item_2do': 'De limón'}
{'seccion': 'Cualquier producto que responda al texto específico de una de las partidas 28.44 o 28.45, se clasifica en dicha partida y no en otra de la nomenclatura, excepto los minerales de metales radiactivos.salvo lo dispuesto en el apartado a) anterior, cu

In [9]:
# Prompt para evaluar seccion
prompt_template = (
    'Eres un asistente preciso y meticuloso que se caracteriza por su excepcional atención al detalle'
    ' y su capacidad para seguir instrucciones al pie de la letra. Posees'
    ' una gran capacidad de comprensión lectora, lo que te permite entender información compleja'
    ' y ejecutar tareas con precisión. Organizado y analítico, te destacas en el mantenimiento del orden'
    ' y la identificación de posibles problemas.\n\n'

    'Un analista solicita tu ayuda para hacer la clasificación arancelaria correcta de una mercadería.'
    ' El analista desea saber si la descripción de la mercadería se ajusta a las notas de la'
    ' nomenclatura arancerlaria oficial de la sección {seccion} (capítulos {first_capitulo} a {last_capitulo},'
    ' "{seccion_description}").\n\n'
    
    'TU RESPUESTA DEBE BASARSE EXCLUSIVAMENTE EN LAS NOTAS DE SECCIÓN PROPORCIONADAS.\n\n' # TODO maybe not?

    '## Descripción de la mercadería:\n\n'
    '{item_description}\n\n'

    '## Notas de la sección {seccion}:\n\n'
    '{seccion_notes}'
)

# TODO agregar algo como " analizar las notas de la sección y compararlas con las características del producto."?
# TODO agregar algo como " determinar si la mercadería parece estar excluida de la sección según las notas."?
    # Incluso tal vez solo pedir que excluya o no excluya?
    # e.g. tal vez un agente que excluye, otro que sugiere.
# TODO agregar algo como " dont use background information"?
# TODO pasar el header del prompt como role system?

In [10]:
if seccion_notes:
    prompt = prompt_template.format(
        seccion=seccion,
        seccion_description=seccion_description,
        item_description=item_description,
        seccion_notes=seccion_notes,
        first_capitulo=first_capitulo,
        last_capitulo=last_capitulo
    )
    print(prompt)
else:
    print("No se encontraron notas para la sección")

Eres un asistente preciso y meticuloso que se caracteriza por su excepcional atención al detalle y su capacidad para seguir instrucciones al pie de la letra. Posees una gran capacidad de comprensión lectora, lo que te permite entender información compleja y ejecutar tareas con precisión. Organizado y analítico, te destacas en el mantenimiento del orden y la identificación de posibles problemas.

Un analista solicita tu ayuda para hacer la clasificación arancelaria correcta de una mercadería. El analista desea saber si la descripción de la mercadería se ajusta a las notas de la nomenclatura arancerlaria oficial de la sección VI (capítulos 28 a 38, "Productos de las industrias químicas o de las industrias conexas").

TU RESPUESTA DEBE BASARSE EXCLUSIVAMENTE EN LAS NOTAS DE SECCIÓN PROPORCIONADAS.

## Descripción de la mercadería:

LIMONES FRESCOS A GRANEL. FRESCOS.

## Notas de la sección VI:

Cualquier producto que responda al texto específico de una de las partidas 28.44 o 28.45, se cl

In [11]:
# prompt siguiente:
prompt_followup = (
    'Finalmente, indica al analista cuál es el código sugerido y'
    ' una breve justificación. Responde en formato JSON con las'
    ' claves "codigo" y "justificacion". TU RESPUESTA DEBE BASARSE EXCLUSIVAMENTE EN LAS'
    ' NOTAS DE SECCIÓN PROPORCIONADAS. '
)
print(prompt_followup)

# TODO buscar manera de determinar si el codigo que sugiere es cap, partida, subp, pos.
# En gral es alguno de esos 4, entonces siempre podriamos quedarnos con los primeros 2 digitos y usar cap.

# TODO una posibilidad debe ser ningun codigo -->
# Idea: un agente que excluye o no, otro que sugiere o no; puede ser suboptimo en relacion 
# a algunos casos donde gpt usa background info, pero puede ser mas seguro (tenemos mas control).

Finalmente, indica al analista cuál es el código sugerido y una breve justificación. Responde en formato JSON con las claves "codigo" y "justificacion". TU RESPUESTA DEBE BASARSE EXCLUSIVAMENTE EN LAS NOTAS DE SECCIÓN PROPORCIONADAS. 


In [12]:
# First message:
messages = [
    # {"role": "system", "content": "..."},
    {"role": "user", "content": prompt},
]
api_response = get_completion(client, messages, logprobs=True, top_logprobs=1, response_format="text")

In [13]:
print(api_response.choices[0].message.content)
# Ejemplo limones: rpta muy mala comparada con ChatGPT web.

De acuerdo con las notas de la sección VI proporcionadas, los limones frescos a granel se clasificarían en la partida correspondiente a esa descripción. No se menciona ninguna partida específica dentro de la sección VI para los limones frescos, por lo tanto, se clasificarían en la partida que corresponda a esa descripción en particular. Es importante tener en cuenta que la descripción de la mercadería debe coincidir exactamente con el texto específico de alguna partida dentro de la nomenclatura arancelaria.


In [15]:
# Second message:
messages = [
    # {"role": "system", "content": "..."},
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": api_response.choices[0].message.content},
    {"role": "user", "content": prompt_followup},
]
api_response_2 = get_completion(client, messages, logprobs=True, top_logprobs=1, response_format="json_object")

In [20]:
json_response = json.loads(api_response_2.choices[0].message.content)
print(json.dumps(json_response, indent=4, ensure_ascii=False))

{
    "codigo": "Sección VI - No hay partida específica mencionada para limones frescos a granel",
    "justificacion": "Según las notas de la sección VI, los limones frescos a granel no responden al texto específico de ninguna partida dentro de esta sección. Por lo tanto, no se puede sugerir un código específico de clasificación en la nomenclatura arancelaria basándose únicamente en la información proporcionada."
}


In [ ]:
# TODO gpt4 might be better than gpt3.5 turbo for this task? (lectura de textos largos)


ETC:

In [20]:
mask = df_items["notas_seccion"].notna()
df_items[mask][["descripcion_seccion", "notas_seccion"]].drop_duplicates()

,descripcion_seccion,notas_seccion
0,animales vivos y productos del reino animal,"En esta Sección, cualquier referencia a un gén..."
517,productos del reino vegetal,"En esta Sección, el término«pellets»designa lo..."
1014,productos de las industrias alimentarias; bebi...,"En esta Sección, el término«pellets»designa lo..."
1540,productos de las industrias químicas o de las ...,Cualquier producto que responda al texto espec...
4627,plástico y sus manufacturas,Los productos presentados en surtidos que cons...
5570,materias textiles y sus manufacturas,Esta Sección no comprende:los pelos y cerdas p...
6978,metales comunes y manufacturas de estos metales,Esta Sección no comprende:los colores y tintas...
7725,"máquinas y aparatos, material eléctrico y sus ...",Esta Sección no comprende:las correas transpor...
9484,material de transporte,Esta Sección no comprende los artículos de las...
